In [46]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder as le
from sklearn.preprocessing import OneHotEncoder as en
from sklearn.impute import SimpleImputer

In [47]:
df = pd.read_csv("/Users/shyam.muralidharan/Downloads/Revlon Dataset Updated Aug 2019.csv")
df.head()

,Channel,Category,Sub Category,Manufacturer,Brand,Revenue,Date,Units,Price,Price Tier
0,MASS,FRAG MASS,WOMEN,CHANEL INC,CHANEL,0.0,8/1/2017,82,0,a.less than $25
1,PRESTIGE,FRAGRANCES,MEN,CHANEL INC,CHANEL,6253422.0,8/1/2017,23,271888,d.$100+
2,PRESTIGE,FRAGRANCES,WOMEN,CHANEL INC,CHANEL,17512844.0,8/1/2017,87,201297,d.$100+
3,PRESTIGE,MAKEUP,MAKEUP TOTAL ALL OTHER,CHANEL INC,CHANEL,-957.0,8/1/2017,58,-17,a.less than $25
4,PRESTIGE,MAKEUP,MAKEUP TOTAL EYE,CHANEL INC,CHANEL,3763488.0,8/1/2017,56,67205,c.$50 - $100


In [45]:
for col in df.columns:
    df[col].fillna('NA',inplace=True)
    if df[col].dtype=='object':
        df[col] = OneHotEncoder(categorical_features = df[col], sparse=False )

NameError: name 'OneHotEncoder' is not defined

In [27]:

df.columns

Index(['Channel', 'Category', 'Sub Category', 'Manufacturer', 'Brand',
       'Revenue', 'Date', 'Units', 'Price', 'Price Tier'],
      dtype='object')

In [30]:
df.head(10)

,Channel,Category,Sub Category,Manufacturer,Brand,Revenue,Date,Units,Price,Price Tier
0,0,6,72,0,0,0.0,20,82,0,0
1,1,7,41,0,0,6253422.0,20,23,271888,3
2,1,7,72,0,0,17512844.0,20,87,201297,3
3,1,11,35,0,0,-957.0,20,58,-17,0
4,1,11,36,0,0,3763488.0,20,56,67205,2
5,1,11,37,0,0,5192224.0,20,23,225749,3
6,1,11,38,0,0,3050499.0,20,75,40673,2
7,1,11,39,0,0,265180.0,20,51,5200,2
8,1,11,40,0,0,97821.0,20,60,1630,0
9,1,15,63,0,0,10255.0,20,71,144,0


In [6]:
#Normalizing the date since every data coulumns with different range points 
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled)


In [7]:
# df_scaled.head()

In [8]:
df_scaled.columns = ["Channel","Category","Sub Category", "Manufacturer", "Brand", "Revenue", "Date", "Units", "Price", "Price Tier"]

In [9]:
# df_scaled.head()

In [10]:
X = df_scaled.drop(["Revenue", "Date","Units","Price","Price Tier"],axis=1).values

X



array([[0.        , 0.4       , 1.        , 0.        , 0.        ],
       [1.        , 0.46666667, 0.56944444, 0.        , 0.        ],
       [1.        , 0.46666667, 1.        , 0.        , 0.        ],
       ...,
       [1.        , 0.73333333, 0.54166667, 0.35714286, 1.        ],
       [1.        , 0.73333333, 0.55555556, 0.35714286, 1.        ],
       [1.        , 1.        , 0.88888889, 0.35714286, 1.        ]])

In [11]:
df_scaled['Revenue'] = df_scaled['Revenue'].apply(lambda x: '1' if x <= 0.05 else '0')
# y = df['Revenue']
# y=y.astype('int')

In [12]:
# df_scaled.head()

In [13]:
y  = df_scaled['Revenue'].values
y = y.ravel(

In [14]:
y

array(['1', '0', '0', ..., '1', '1', '1'], dtype=object)

In [15]:
forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', n_estimators=1000)

In [16]:
# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2)


In [17]:
# find all relevant features 
feat_selector.fit(X, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	5
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	1


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	0
Rejected: 	1


BorutaPy(alpha=0.05,
         estimator=RandomForestClassifier(bootstrap=True,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=None,
                                          max_features='auto',
                                          max_leaf_nodes=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=31, n_jobs=-1,
                                          oob_score=False,
                                          random_state=<mtrand.RandomState object at 0x10596eb40>,
                                          verbose=0, warm_start=False),
         max_iter=1

In [18]:
feat_selector.support_


array([False,  True,  True,  True,  True])

In [19]:
feat_selector.ranking_


array([2, 1, 1, 1, 1])

In [56]:
new = {
    "fruit": "apple",
    "date" : "23",
    "sold" : "35"
},
{
   "fruit": "apple",
    "date" : "233",
    "sold" : "354" 
}

{'fruit': 'apple', 'date': '233', 'sold': '354'}

In [57]:
le = []
for key, value in new.item():
    if key == 'friut':
        le.append(value)

AttributeError: 'tuple' object has no attribute 'item'